## RoboQuery: Retrieval-Augmented Generation (RAG) Service

### Introduction

RoboQuery is an Internal Knowledge Base Assistant intended for employees of XXX company to ask questions regarding internal company policies.

RoboQuery scans through the company's document repository on Google Drive to provide an accurate answer efficiently.

### Problem Statement

Employees may not remember minute details about company policies or procedures and have to manually sift through many documents to retrieve this information.

RoboQuery eliminates this manual effort, allowing employees to quickly retrieve needed information and spend their time more productively.

### Technical Architecture

[add visio]

### Setup

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import GoogleDriveLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma

# from langchain_community.embeddings import HuggingFaceEmbeddings (deprecated)

### Configuration

In [ ]:
FOLDER_ID = "1M86nx_y-v-uWc9c-EylmuxJTaoZ8bcca"
DB_PATH = "chroma_db"
EMBEDDING_MODEL = "all-MiniLM-L6-v2"
LLM_MODEL = "llama-3.3-70b-versatile"
DISTANCE_THRESHOLD = 0.9
FAILED_RESPONSE = "This information is not present in any internal document."

### Load Documents from Google Drive

In [67]:
loader = GoogleDriveLoader(folder_id=FOLDER_ID)
documents = loader.load()
print(f"Loaded {len(documents)} documents from Google Drive: {documents}")

Loaded 3 documents from Google Drive: [Document(metadata={'source': 'https://docs.google.com/document/d/1uT48fUsbTe-mtBizGYcQ89DSpZYpUEf98rk36K38HJg/edit', 'title': 'Employee PDPA Guidelines', 'when': '2026-01-13T16:05:48.896Z'}, page_content='\ufeffEmployee PDPA Guidelines\r\nVersion: 2.1 | Last Updated: November 2025\r\n\r\n\r\n1. Definition of Personal Data\r\nUnder the Singapore Personal Data Protection Act (PDPA), personal data includes NRIC numbers, mobile numbers, and residential addresses.\r\n\r\n\r\n2. NRIC Handling\r\nIn accordance with PDPC guidelines, the company will not collect full NRIC numbers unless required by law (e.g., for CPF, IRAS, or security clearance at the office building). For internal identifiers, only the last 4 characters (e.g., ###567A) may be used.\r\n\r\n\r\n3. Reporting Breaches\r\nIf you suspect a data leak (e.g., sending a client list to the wrong email), you must report it to our Data Protection Officer (DPO) at dpo@lioncitytech.com within 24 hours.

### Split Documents into Chunks

In [69]:
splitter = RecursiveCharacterTextSplitter(
        chunk_size=300,
        chunk_overlap=50
    )
chunks = splitter.split_documents(documents)
print(f"Split documents into {len(chunks)} chunks")

Split documents into 11 chunks


### Create Embeddings and Store in Chroma DB

In [70]:
# convert chunks to vectors (embeddings)
# vectors preserve semantic meaning: similar meaning = closer vectors (cosine similarity)
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

# store in vector DB (vector -> text chunk -> metadata) so internal docs are queryable by semantics
vectorstore = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory=DB_PATH
)

print("Documents ingested successfully into Chroma")

Documents ingested successfully into Chroma


## 2. Set up LLM and Prompt Template

In [ ]:
load_dotenv()

llm = ChatGroq(
    temperature=0,
    api_key=os.getenv("LLM_API_KEY"),
    model=LLM_MODEL
)

prompt_template = PromptTemplate.from_template(
    """
    You are an internal company knowledge assistant.
    Answer questions ONLY using the provided context.
    Always cite the source document names.
    If the answer is not in the context, say {failed_response}.
    Do not cite any sources in that case.
    
    Context: {context}
    
    Question: {question}"""
)

### Query Processing

In [72]:
embeddings = HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL)

vectorstore = Chroma(
        persist_directory=DB_PATH,
        embedding_function=embeddings
    )

def answer_question(question: str):
    distances = vectorstore.similarity_search_with_score(question, k=3)

    # keep only documents whose distance is small enough (semantically similar to question)
    relevant_docs = [
        doc for doc, distance in distances
        if distance < DISTANCE_THRESHOLD
    ]

    if not relevant_docs:
        return FAILED_RESPONSE

    context = "\n\n".join(
        f"Source: {doc.metadata['source']}\n{doc.page_content}"
        for doc in relevant_docs
    )

    prompt = prompt_template.format(
        failed_response=FAILED_RESPONSE,
        context=context,
        question=question)
    
    response = llm.invoke(prompt)
    return response.content

### StreamLit UI

In [59]:
import streamlit as st

st.set_page_config(page_title="Internal Knowledge Assistant")

st.title("📄 Internal Knowledge Base Assistant")

question = st.text_input("Ask a question about company policies:")

if question:
    with st.spinner("Searching knowledge base..."):
        answer = answer_question(question)
        st.markdown("### Answer")
        st.write(answer)


2026-01-14 03:39:56.496 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 03:39:56.499 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 03:39:56.500 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 03:39:56.501 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 03:39:56.502 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 03:39:56.503 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 03:39:56.503 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2026-01-14 03:39:56.504 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar